In [1]:
import psutil
import os
print("Starting ResNet Weight Steganalysis Research")
print(f"Initial memory: {psutil.Process(os.getpid()).memory_info().rss / 1024 / 1024:.2f} MB")

Starting ResNet Weight Steganalysis Research
Initial memory: 63.25 MB


In [2]:
# 1. Load model dan ekstrak bobot
from ResNetWeightExtractorX import ResNetWeightExtractorX
modelSelected = 'resnet18'
print("Loading model...")
extractorX = ResNetWeightExtractorX(f'../data/models/{modelSelected}.pth')

Loading model...
Loaded state_dict directly
Total keys in state_dict: 122


In [3]:
print(extractorX)

In [5]:
# Lihat statistik layer
stats = extractorX.get_layer_statistics()
print("\nLayer statistics (first 10 layers):")
for i, (name, stat) in enumerate(list(stats.items())[:10]):
    print(f"{name}: shape{stat['shape']}, elements: {stat['numel']}")


Layer statistics (first 10 layers):
conv1.weight: shapetorch.Size([64, 3, 7, 7]), elements: 9408
bn1.weight: shapetorch.Size([64]), elements: 64
bn1.bias: shapetorch.Size([64]), elements: 64
bn1.running_mean: shapetorch.Size([64]), elements: 64
bn1.running_var: shapetorch.Size([64]), elements: 64
bn1.num_batches_tracked: shapetorch.Size([]), elements: 1
layer1.0.conv1.weight: shapetorch.Size([64, 64, 3, 3]), elements: 36864
layer1.0.bn1.weight: shapetorch.Size([64]), elements: 64
layer1.0.bn1.bias: shapetorch.Size([64]), elements: 64
layer1.0.bn1.running_mean: shapetorch.Size([64]), elements: 64


In [6]:
# Opsi 1: Ekstrak semua bobot convolutional
print("\n=== Extracting all convolutional weights ===")
conv_weights, layer_names = extractorX.extract_conv_weights_only()


=== Extracting all convolutional weights ===
Extracting: conv1.weight, Shape: torch.Size([64, 3, 7, 7])
Extracting: layer1.0.conv1.weight, Shape: torch.Size([64, 64, 3, 3])
Extracting: layer1.0.conv2.weight, Shape: torch.Size([64, 64, 3, 3])
Extracting: layer1.1.conv1.weight, Shape: torch.Size([64, 64, 3, 3])
Extracting: layer1.1.conv2.weight, Shape: torch.Size([64, 64, 3, 3])
Memory usage: 251.16 MB
Extracting: layer2.0.conv1.weight, Shape: torch.Size([128, 64, 3, 3])
Extracting: layer2.0.conv2.weight, Shape: torch.Size([128, 128, 3, 3])
Extracting: layer2.0.downsample.0.weight, Shape: torch.Size([128, 64, 1, 1])
Extracting: layer2.1.conv1.weight, Shape: torch.Size([128, 128, 3, 3])
Extracting: layer2.1.conv2.weight, Shape: torch.Size([128, 128, 3, 3])
Memory usage: 253.17 MB
Extracting: layer3.0.conv1.weight, Shape: torch.Size([256, 128, 3, 3])
Extracting: layer3.0.conv2.weight, Shape: torch.Size([256, 256, 3, 3])
Extracting: layer3.0.downsample.0.weight, Shape: torch.Size([256, 128

In [7]:
if not conv_weights:
    print("No convolutional weights found! Trying alternative extraction...")
    # Fallback: ekstrak semua weights yang ada
    conv_weights = []
    for name, param in extractorX.state_dict.items():
        if 'weight' in name and len(param.shape) >= 2:  # Filter kemungkinan conv weights
            weight_vec = param.cpu().numpy().flatten()
            conv_weights.append(weight_vec)
            print(f"Fallback extraction: {name}, {len(weight_vec)} weights")

In [8]:
print(f"\nSuccessfully extracted {len(conv_weights)} weight arrays")


Successfully extracted 20 weight arrays


In [9]:
print(f"\nExtracted {len(conv_weights)} convolutional layers")


Extracted 20 convolutional layers


In [11]:
# 3. Persiapkan data training dengan parameter yang benar
print("\n=== Preparing training data ===")

# Gunakan hanya 2-3 layer pertama untuk menghemat memory
training_weights = conv_weights[:3] if len(conv_weights) >= 3 else conv_weights
        
# Buat dataset dengan parameter yang sesuai
from SteganalysisAutoencoder import WeightDataset
dataset = WeightDataset(
    weight_data=training_weights,
    seq_length=128,  # Sequence length lebih kecil
    max_samples=2000  # Batasi jumlah samples
)

if len(dataset) == 0:
    # Fallback: coba dengan seq_length yang lebih kecil
    print("No data with seq_length=128, trying with smaller seq_length...")
    dataset = WeightDataset(
        weight_data=training_weights,
        seq_length=64,  # Lebih kecil
        max_samples=2000
    )
if len(dataset) == 0:
    raise ValueError("No training data available even with smaller seq_length!")



=== Preparing training data ===
Processing weight data with max_samples: 2000
Weight array 0: 9408 weights -> 73 sequences
Weight array 1: 36864 weights -> 288 sequences
Weight array 2: 36864 weights -> 288 sequences
Dataset created: 649 sequences of length 128


In [12]:
# DataLoader dengan batch size sangat kecil
from torch.utils.data import DataLoader
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=0)
print(f"Training samples: {len(dataset)}")
print(f"Batch size: 4")
print(f"Memory before training: {psutil.Process(os.getpid()).memory_info().rss / 1024 / 1024:.2f} MB")

Training samples: 649
Batch size: 4
Memory before training: 294.26 MB


In [13]:
# 4. Inisialisasi model kecil
input_dim = dataset[0][0].shape[0]
print(f"Input dimension: {input_dim}")

Input dimension: 128


In [14]:
from SteganalysisAutoencoder import SteganalysisAutoencoder
autoencoder = SteganalysisAutoencoder(input_dim=input_dim, encoding_dim=8)
print(f"Model parameters: {sum(p.numel() for p in autoencoder.parameters()):,}")

Model parameters: 21,320


#### Start Training

In [ ]:

print("\n=== Starting Training ===")
from Training import train_with_progress, train_with_memory_monitoring, simple_train
# Pilih salah satu metode training:
trained_model = train_with_memory_monitoring(autoencoder, dataloader, epochs=20, modelSelected=modelSelected)
# trained_model = simple_train(autoencoder, dataloader, epochs=2)
# trained_model = train_with_progress(autoencoder, dataloader, epochs=20)


=== Starting Training ===
Epoch 0, Batch 0, Loss: 0.739480, Memory: 352.35 MB
Epoch 0, Batch 10, Loss: 0.815997, Memory: 352.37 MB
Epoch 0, Batch 20, Loss: 0.756442, Memory: 352.37 MB
Epoch 0, Batch 30, Loss: 0.800024, Memory: 352.40 MB
Epoch 0, Batch 40, Loss: 0.733418, Memory: 352.36 MB
Epoch 0, Batch 50, Loss: 0.797735, Memory: 352.36 MB
Epoch 0, Batch 60, Loss: 0.870083, Memory: 352.36 MB
Epoch 0, Batch 70, Loss: 0.781786, Memory: 352.36 MB
Epoch 0, Batch 80, Loss: 0.775986, Memory: 352.36 MB
Epoch 0, Batch 90, Loss: 0.815807, Memory: 352.36 MB
Epoch 0, Batch 100, Loss: 0.704798, Memory: 352.36 MB
Epoch 0, Batch 110, Loss: 0.778827, Memory: 352.36 MB
Epoch 0, Batch 120, Loss: 0.645098, Memory: 352.36 MB
Epoch 0, Batch 130, Loss: 0.805736, Memory: 352.36 MB
Epoch 0, Batch 140, Loss: 0.627133, Memory: 352.36 MB
Epoch 0, Batch 150, Loss: 0.799716, Memory: 352.36 MB
Epoch 0, Batch 160, Loss: 0.816288, Memory: 352.36 MB
Epoch 0 completed. Average Loss: 0.793477
Epoch 1, Batch 0, Loss: 

### # Simpan model

In [29]:
import torch
import torch.nn as nn
torch.save(trained_model.state_dict(), f'../data/models/{modelSelected}_autoencoder_final.pth')
print("Model saved successfully!")

Model saved successfully!


### # Test reconstruction

In [30]:

print("\n=== Testing Reconstruction ===")
trained_model.eval()
with torch.no_grad():
    test_sample, _ = dataset[0]
    test_sample = test_sample.unsqueeze(0)
    encoded, decoded = trained_model(test_sample)
    test_loss = nn.MSELoss()(decoded, test_sample)
    print(f"Test reconstruction loss: {test_loss.item():.6f}")
        
print("Research completed successfully!")


=== Testing Reconstruction ===
Test reconstruction loss: 0.750238
Research completed successfully!
